# Datasheet Analyzer Setup

이 노트북은 Datasheet Analyzer를 Google Colab에서 실행하기 위한 설정을 제공합니다.

In [ ]:
# GPU 사용 가능 여부 확인
!nvidia-smi

# PyTorch가 GPU를 인식하는지 확인
import torch
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"현재 사용 중인 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")

In [ ]:
# Clone the repository
!git clone https://github.com/doyoung42/autodatasheet.git
%cd autodatasheet

In [ ]:
# Install dependencies
!pip install -r requirements.txt

# Install additional dependencies for Google Colab
!pip install streamlit-option-menu

In [ ]:
# Python 패키지 경로 설정
import sys
import os
from pathlib import Path

# 현재 작업 디렉토리를 Python 경로에 추가
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# src 디렉토리를 Python 경로에 추가
src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    # config, models, utils 디렉토리도 추가
    for subdir in ['config', 'models', 'utils']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

print("Python 경로:")
for path in sys.path:
    print(f"- {path}")

# 토큰 설정
from config.token_manager import TokenManager
from getpass import getpass

token_manager = TokenManager()

# OpenAI API 토큰 설정
openai_token = getpass('OpenAI API 토큰을 입력하세요: ')
if openai_token:
    token_manager.set_token('openai', openai_token)
    print('OpenAI API 토큰이 저장되었습니다.')

# HuggingFace API 토큰 설정
hf_token = getpass('HuggingFace API 토큰을 입력하세요: ')
if hf_token:
    token_manager.set_token('huggingface', hf_token)
    print('HuggingFace API 토큰이 저장되었습니다.')

# Anthropic API 토큰 설정
anthropic_token = getpass('Anthropic API 토큰을 입력하세요: ')
if anthropic_token:
    token_manager.set_token('anthropic', anthropic_token)
    print('Anthropic API 토큰이 저장되었습니다.')

In [ ]:
# Python 패키지 경로 설정
import sys
import os
from pathlib import Path

# 현재 작업 디렉토리를 Python 경로에 추가
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# src 디렉토리를 Python 경로에 추가
src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    # config, models, utils 디렉토리도 추가
    for subdir in ['config', 'models', 'utils']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

print("Python 경로:")
for path in sys.path:
    print(f"- {path}")


# 로깅 설정
import logging
import os
import sys
from pathlib import Path

# 로그 디렉토리 생성
log_dir = Path('/tmp/streamlit_logs')
log_dir.mkdir(exist_ok=True, parents=True)
log_file = log_dir / 'streamlit.log'

# 로깅 설정
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(log_file)
    ]
)
logger = logging.getLogger(__name__)

# Streamlit 실행
import subprocess
import threading
import time
import traceback

def run_streamlit():
    try:
        logger.info("Starting Streamlit server...")
        # 환경 변수 설정
        env = os.environ.copy()
        env['STREAMLIT_SERVER_HEADLESS'] = 'true'
        env['STREAMLIT_SERVER_PORT'] = '8501'
        env['STREAMLIT_SERVER_ADDRESS'] = '0.0.0.0'
        env['STREAMLIT_BROWSER_SERVER_ADDRESS'] = '0.0.0.0'
        env['STREAMLIT_LOGGER_LEVEL'] = 'debug'
        env['STREAMLIT_SERVER_MAX_UPLOAD_SIZE'] = '200'
        env['STREAMLIT_SERVER_ENABLE_CORS'] = 'false'
        env['STREAMLIT_SERVER_ENABLE_XSRF_PROTECTION'] = 'false'
        
        # Python 경로 설정
        current_dir = Path(os.getcwd())
        python_path = f"{current_dir}:{current_dir}/src"
        env['PYTHONPATH'] = python_path
        
        # Streamlit 실행
        process = subprocess.Popen([
            "streamlit", "run",
            "src/app/streamlit_app.py",
            "--server.port=8501",
            "--server.address=0.0.0.0",
            "--browser.serverAddress=0.0.0.0",
            "--server.maxUploadSize=200",
            "--server.enableCORS=false",
            "--server.enableXsrfProtection=false"
        ], env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE)        
        
        # 로그 출력
        while True:
            output = process.stdout.readline()
            if output:
                print(output.decode().strip())
            error = process.stderr.readline()
            if error:
                print(error.decode().strip())
            if process.poll() is not None:
                break
                
    except Exception as e:
        logger.error(f"Error running Streamlit: {str(e)}")
        print(traceback.format_exc())

# 백그라운드에서 Streamlit 실행
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# 서버 시작 대기
time.sleep(10)  # 대기 시간을 10초로 증가

# 포트 포워딩
from google.colab.output import eval_js
print("Streamlit URL:")
print(eval_js("google.colab.kernel.proxyPort(8501)"))

In [ ]:
# 서버 상태 확인
!ps aux | grep streamlit
!netstat -tulpn | grep 8501

In [ ]:
# Streamlit 로그 확인
import time
from pathlib import Path

def check_streamlit_logs():
    """Streamlit 로그를 확인하는 함수"""
    try:
        log_file = Path('/tmp/streamlit_logs/streamlit.log')
        
        if log_file.exists():
            print(f"로그 파일을 찾았습니다: {log_file}")
            # 로그 파일 실시간 모니터링
            with open(log_file, 'r') as f:
                # 파일 끝으로 이동
                f.seek(0, 2)
                while True:
                    line = f.readline()
                    if line:
                        print(line.strip())
                    time.sleep(0.1)
        else:
            print(f"로그 파일을 찾을 수 없습니다: {log_file}")
            print("\n현재 실행 중인 Streamlit 프로세스 확인:")
            !ps aux | grep streamlit
            
            print("\n/tmp/streamlit_logs 디렉토리 확인:")
            !ls -la /tmp/streamlit_logs
            
            print("\n현재 디렉토리 확인:")
            !pwd
            !ls -la
    except Exception as e:
        print(f"로그 확인 중 오류 발생: {str(e)}")
        import traceback
        print(traceback.format_exc())

# 로그 확인 시작
print("Streamlit 로그 확인을 시작합니다...")
check_streamlit_logs()